# Generating Generic PSFs with `spike`

Some of the utility of `spike` is baked into the fact that it creates PSFs for individual images/coordinates using the consistent syntax across a variety of different PSF generation methods. Even outside of cases where a drizzled/resampled PSF would be useful, these image-specific PSFs can be generated quickly and easily.

This example notebook assumes that you have all of the optional PSF generation dependencies installed -- i.e., `TinyTim`, `PSFEx` (and `SExtractor`), and `WebbPSF`. It uses the following _HST_ and _JWST_ files (also used in generating the Polzin 2025 figures and in the other example notebooks):
- j8pu42ecq_flc.fits (_HST_/ACS; observation ID j8pu42010)
- jw02514162001_03201_00001_nrca2_cal.fits (_JWST_/NIRCam; observation ID jw02514162001_03201_00001_nrca2)

Other than using paths for these data, the syntax is generic and servers as a guide for use with other data.

In [1]:
from spike import psfgen
from spike.tools import objloc, checkpixloc